In [14]:
#
#
# USANDO O SVM NA BASE DE DADOS DO DIABETES
# ESCOLHENDO VARIOS Kernels PARA A BASE
#

import pandas
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
import matplotlib.colors as mcolors
import os
# Importer ces fonctions pour des calculs de métriques spécifiques
from sklearn.metrics import confusion_matrix, classification_report

from math import ceil
plt.rcParams['figure.figsize'] = [16, 10]

#definindo os nomes de cada coluna
names = ['num-pregnant', 'glucose', 'diastolic', 'triceps-skin', 'insulin', 'body-mass', 'diabetes-pedigree', 'age', 'class']

dataset = pandas.read_csv("/Users/nathanjacas/Documents/Cours/AM/Avaliaçao Comparativa/dataset.csv")

#########################################################################
######                     Preparação de Dados                  #########
#########################################################################



#Limpeza do data frame de dados faltantes
dataset = dataset.dropna(axis=0, how='any')

#print("Primeiros dados")
#print(dataset.head(5))

#Criação de um dataframe menor sem a ultima coluna
datasetfeat = dataset.iloc[:, 1:-1]

#Eliminação das colunas com dados não numéricos
dataset1 = datasetfeat.select_dtypes(include=['int64', 'float64'])

df_max = dataset1.max(axis=0)
df_min = dataset1.min(axis=0)
#print('Max:',df_max)
#print('Min:',df_min)

#Ajuste de escala
for coluna in dataset1.columns:
    dataset1[coluna] = (dataset1[coluna] - df_min[coluna]) / (df_max[coluna] - df_min[coluna])

# Exibindo o DataFrame
print(dataset1.head(5))


#divisao de dados atributos e classe
X = dataset1.values[:,:]  # 0:-1] #caracteristicas (features)
Y = dataset.values[:, -1] #classe a ser analisada: track_genre

#Utilizar o 50% dos dados
print('Tamanho do vetor:',len(Y))
d50 = int(0.50*len(Y))

X = X[0:d50,:]
Y = Y[0:d50]

print('Tamanho do vetor 50%',len(Y))


print(X.shape[0],X.shape[1])




#construindo conjuntos de treinamento, validacao e teste
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.25, random_state = 10)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.25, random_state = 10)

   popularity  duration_ms  danceability  energy       key  loudness  mode  \
0        0.73     0.042473      0.686294  0.4610  0.090909  0.791392   0.0   
1        0.55     0.026971      0.426396  0.1660  0.090909  0.597377   1.0   
2        0.57     0.038679      0.444670  0.3590  0.000000  0.736123   1.0   
3        0.71     0.036978      0.270051  0.0596  0.000000  0.573701   1.0   
4        0.82     0.036389      0.627411  0.4430  0.181818  0.737103   1.0   

   speechiness  acousticness  instrumentalness  liveness   valence     tempo  \
0     0.148187      0.032329          0.000001    0.3580  0.718593  0.361245   
1     0.079067      0.927711          0.000006    0.1010  0.268342  0.318397   
2     0.057720      0.210843          0.000000    0.1170  0.120603  0.313643   
3     0.037617      0.908635          0.000071    0.1320  0.143719  0.746758   
4     0.054508      0.470884          0.000000    0.0829  0.167839  0.492863   

   time_signature  
0             0.8  
1         

In [15]:

#########################################################################
######                     Decision Tree                        #########
#########################################################################
import os
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# Configuration de la validation croisée stratifiée
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)

# Définir le modèle d'arbre de décision avec critère d'entropie
clf = DecisionTreeClassifier(criterion='entropy', random_state=10)

# Définir les métriques à évaluer
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='weighted'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, multi_class="ovr")
}

# Exécution de la validation croisée
results = cross_validate(clf, X_train, y_train, cv=cv, scoring=scoring, return_train_score=False, n_jobs=-1)

# Formatage et sauvegarde des résultats
results_df = pd.DataFrame()
file_path = '/Users/nathanjacas/Documents/Cours/AM/Avaliaçao Comparativa/Comparaison_utility/model_performance.csv'
for metric, scores in results.items():
    if 'test_' in metric:
        for fold_index, score in enumerate(scores):
            new_row = pd.DataFrame({
                'Model': ['Decision Tree'],  # Changement du modèle pour 'Decision Tree'
                'Metric': [metric.split('test_')[1]],
                'Fold': [fold_index + 1],
                'Score': [score]
            })
            results_df = pd.concat([results_df, new_row], ignore_index=True)

# Vérification de l'existence du fichier et de son contenu
header = not os.path.exists(file_path) or os.stat(file_path).st_size == 0

# Écriture dans le fichier CSV
results_df.to_csv(file_path, mode='a', header=header, index=False)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


In [16]:
#########################################################################
######                     SVM                                  #########
#########################################################################
import os
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# Configuration de la validation croisée stratifiée
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)

# Définir le modèle SVM avec le kernel poly
clf = SVC(C=30, kernel='poly', probability=True, random_state=10)

# Définir les métriques à évaluer
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='weighted'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, multi_class="ovr")
}

# Exécution de la validation croisée
results = cross_validate(clf, X_train, y_train, cv=cv, scoring=scoring, return_train_score=False, n_jobs=-1)

# Formatage et sauvegarde des résultats
results_df = pd.DataFrame()
file_path = '/Users/nathanjacas/Documents/Cours/AM/Avaliaçao Comparativa/Comparaison_utility/model_performance.csv'
for metric, scores in results.items():
    if 'test_' in metric:
        for fold_index, score in enumerate(scores):
            new_row = pd.DataFrame({
                'Model': ['SVM'],
                'Metric': [metric.split('test_')[1]],
                'Fold': [fold_index + 1],
                'Score': [score]
            })
            results_df = pd.concat([results_df, new_row], ignore_index=True)

# Check if the file exists and has content
header = not os.path.exists(file_path) or os.stat(file_path).st_size == 0

# Writing to CSV
results_df.to_csv(file_path, mode='a', header=header, index=False)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
#########################################################################
######                        KNN                               #########
#########################################################################
import os
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# Configuration de la validation croisée stratifiée
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)

# Définir le modèle KNN
clf = KNeighborsClassifier(n_neighbors=5)  # Vous pouvez ajuster le nombre de voisins si nécessaire

# Définir les métriques à évaluer
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='weighted'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, multi_class="ovr")
}

# Exécution de la validation croisée
results = cross_validate(clf, X_train, y_train, cv=cv, scoring=scoring, return_train_score=False, n_jobs=-1)

# Formatage et sauvegarde des résultats
results_df = pd.DataFrame()
file_path = '/Users/nathanjacas/Documents/Cours/AM/Avaliaçao Comparativa/Comparaison_utility/model_performance.csv'
for metric, scores in results.items():
    if 'test_' in metric:
        for fold_index, score in enumerate(scores):
            new_row = pd.DataFrame({
                'Model': ['KNN'],
                'Metric': [metric.split('test_')[1]],
                'Fold': [fold_index + 1],
                'Score': [score]
            })
            results_df = pd.concat([results_df, new_row], ignore_index=True)

# Vérification de l'existence du fichier et de son contenu
header = not os.path.exists(file_path) or os.stat(file_path).st_size == 0

# Écriture dans le fichier CSV
results_df.to_csv(file_path, mode='a', header=header, index=False)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


In [ ]:
#########################################################################
######                        MLP                               #########
#########################################################################
import os
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# Configuration de la validation croisée stratifiée
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)

# Définir le modèle MLP
clf = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=10)

# Définir les métriques à évaluer
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='weighted'),
    # ROC AUC peut nécessiter des modifications pour les classes multiples ou des approches spécifiques
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, multi_class="ovr")
}

# Exécution de la validation croisée
results = cross_validate(clf, X_train, y_train, cv=cv, scoring=scoring, return_train_score=False, n_jobs=-1)

# Formatage et sauvegarde des résultats
results_df = pd.DataFrame()
file_path = '/Users/nathanjacas/Documents/Cours/AM/Avaliaçao Comparativa/Comparaison_utility/model_performance.csv'  # Modifiez le chemin selon votre environnement
for metric, scores in results.items():
    if 'test_' in metric:
        for fold_index, score in enumerate(scores):
            new_row = pd.DataFrame({
                'Model': ['MLP'],
                'Metric': [metric.split('test_')[1]],
                'Fold': [fold_index + 1],
                'Score': [score]
            })
            results_df = pd.concat([results_df, new_row], ignore_index=True)

# Check if the file exists and has content
header = not os.path.exists(file_path) or os.stat(file_path).st_size == 0

# Writing to CSV
results_df.to_csv(file_path, mode='a', header=header, index=False)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_mult

In [ ]:
import numpy as np
import pandas
from random import seed, randrange
from csv import reader
from math import sqrt
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import make_scorer,roc_auc_score
from sklearn.model_selection import StratifiedKFold

plt.rcParams['figure.figsize'] = [16, 10]

# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual))

# Calculate precision
def precision_metric(actual, predicted):
    true_positives = sum((act == 1 and pred == 1) for act, pred in zip(actual, predicted))
    false_positives = sum((act == 0 and pred == 1) for act, pred in zip(actual, predicted))

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
    return precision

# Calculate recall
def recall_metric(actual, predicted):
    true_positives = sum((act == 1 and pred == 1) for act, pred in zip(actual, predicted))
    false_negatives = sum((act == 1 and pred == 0) for act, pred in zip(actual, predicted))

    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
    return recall

# Calculate F1 Score
def f1_score_metric(actual, predicted):
    true_positives = sum((act == 1 and pred == 1) for act, pred in zip(actual, predicted))
    false_positives = sum((act == 0 and pred == 1) for act, pred in zip(actual, predicted))
    false_negatives = sum((act == 1 and pred == 0) for act, pred in zip(actual, predicted))

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0

    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    return f1

# Calculate ROC AUC
def roc_auc_metric(actual, predicted):
    total_positive = sum(1 for val in actual if val == 1)
    total_negative = sum(1 for val in actual if val == 0)

    # Criar uma lista de tuplas (predicted_probability, true_class)
    predictions = list(zip(predicted, actual))

    # Classificar as previsões com base na probabilidade prevista (em ordem decrescente)
    predictions_sorted = sorted(predictions, key=lambda x: x[0], reverse=True)

    # Inicializar os contadores para TPR e FPR
    true_positive_count = 0
    false_positive_count = 0

    # Inicializar os valores anteriores para o cálculo do AUC
    prev_true_positive_count = 0
    prev_false_positive_count = 0

    # Inicializar a área sob a curva (AUC)
    auc = 0.0

    # Iterar sobre as previsões classificadas
    for pred_prob, true_class in predictions_sorted:
        # Atualizar os contadores TPR e FPR
        if true_class == 1:
            true_positive_count += 1
        else:
            false_positive_count += 1

        # Calcular as taxas TPR e FPR
        tpr = true_positive_count / total_positive
        fpr = false_positive_count / total_negative

        # Calcular a contribuição para o AUC usando a regra trapezoidal
        auc += ((fpr - prev_false_positive_count) * (tpr + prev_true_positive_count)) / 2

        # Atualizar os valores anteriores
        prev_true_positive_count = tpr
        prev_false_positive_count = fpr

    if auc>1 :
        auc = 1
    return auc


def evaluate_algorithm(dataset, algorithm, n_splits, *args):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    rocauc_scores = []
    
    dataset = np.array(dataset)  # Convertir dataset en tableau numpy
    
    for train_index, test_index in skf.split(dataset[:, :-1], dataset[:, -1]):  # Utiliser les caractéristiques et les étiquettes séparément
        train_set = dataset[train_index]
        test_set = dataset[test_index]
        
        train_set_X = train_set[:, :-1]
        train_set_y = train_set[:, -1]
        test_set_X = test_set[:, :-1]
        test_set_y = test_set[:, -1]
        
        predicted = algorithm(train_set, test_set, *args)
        
        accuracy = accuracy_metric(test_set_y, predicted)
        precision = precision_metric(test_set_y, predicted)
        recall = recall_metric(test_set_y, predicted)
        f1 = f1_score_metric(test_set_y, predicted)
        rocaux = roc_auc_metric(test_set_y, predicted)
        
        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        rocauc_scores.append(rocaux)
        
    return accuracy_scores, precision_scores, recall_scores, f1_scores, rocauc_scores

# LVQ Algorithm
def learning_vector_quantization(train, test, n_codebooks, lrate, epochs):
    codebooks = train_codebooks(train, n_codebooks, lrate, epochs)
    predictions = list()
    for row in test:
        output = predict(codebooks, row)
        predictions.append(output)
    return predictions

# calculate the Euclidean distance between two vectors
# calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
	distance = 0.0
	av= 0.0
	for i in range(len(row1)-1):
	  distance = distance + (row1[i] - row2[i])**2
	return sqrt(distance)


# Locate the best matching unit
def get_best_matching_unit(codebooks, test_row):
    distances = [(codebook, euclidean_distance(codebook, test_row)) for codebook in codebooks]
    distances.sort(key=lambda tup: tup[1])
    return distances[0][0]

# Make a prediction with codebook vectors
def predict(codebooks, test_row):
    bmu = get_best_matching_unit(codebooks, test_row)
    return bmu[-1]

# Create a random codebook vector
def random_codebook(train):
    n_records = len(train)
    n_features = len(train[0])
    return [train[randrange(n_records)][i] for i in range(n_features)]

# Train a set of codebook vectors
def train_codebooks(train, n_codebooks, lrate, epochs):
    codebooks = [random_codebook(train) for _ in range(n_codebooks)]
    for epoch in range(epochs):
        rate = lrate * (1.0-(epoch/float(epochs)))
        for row in train:
            bmu = get_best_matching_unit(codebooks, row)
            for i in range(len(row)-1):
                error = row[i] - bmu[i]
                if bmu[-1] == row[-1]:
                    bmu[i] += rate * error
                else:
                    bmu[i] -= rate * error
    return codebooks


#######################################################
######          Preparação de Dados           #########
#######################################################

seed(1)

# Carregar a base de dados
filename = "/Users/nathanjacas/Documents/Cours/AM/Avaliaçao Comparativa/dataset.csv"
dataset0 = pandas.read_csv(filename)


#Limpeza do data frame de dados faltantes
dataset0 = dataset0.dropna(axis=0, how='any')

#Criação de um dataframe menor sem a primeira e ultima coluna
datasetfeat = dataset0.iloc[:, 1:-1]

#Eliminação das colunas com dados não numéricos
dataset2 = datasetfeat.select_dtypes(include=['int64', 'float64'])

df_max = dataset2.max(axis=0)
df_min = dataset2.min(axis=0)

#Adimensionalização das features
for coluna in dataset2.columns:
    dataset2[coluna] = dataset2[coluna] / (df_max[coluna] - df_min[coluna])

#divisao de dados atributos e classe
X = dataset2.values[:,:]  # 0:-1] #caracteristicas (features)
Y = dataset0.values[:, -1] #classe a ser analisada: track_genre

#Utilizar o 100% dos dados
print('Tamanho do vetor:',len(Y))
d50 = int(len(Y))

X = X[0:d50,:]
Y = Y[0:d50]

print('Tamanho do vetor 100%',len(Y))

#Criação de conjuntos de treinamento, validacao e teste
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.40, random_state = 10)

X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.50, random_state = 10)


#Conjunto de Treinamento
#Juntar a matriz de features com o vetor de classes, o vetor de classes é posicionado no final
dataset3 = np.hstack((X_train, y_train[:, np.newaxis]))

#Transforma o array Python para uma lista
dataset4 = dataset3.tolist()

#Transforma os valores da lista em arrayas
dataset =[]
for valores in dataset4:
  arrays = list(map(str, valores))
  dataset.append(arrays)

#Transforma os elementos numéricos da lista em números reais
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)

# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)

#######################################################
######       Fim da Preparação de Dados       #########
#######################################################


#######################################################
######          Avaliação do algoritmo LVQ    #########
#######################################################


scores_nfolds = {'nfolds':[],'accuracy': [], 'precision': [], 'recall': [], 'f1_score': [], 'Roc_Auc':[]}

n_folds = 5
learn_rate = 0.1
n_epochs = 1
n_codebooks = 30 #15

accuracy_scores, precision_scores, recall_scores, f1_scores,rocauc_scores = evaluate_algorithm(dataset, learning_vector_quantization, n_folds, n_codebooks, learn_rate, n_epochs)

print(accuracy_scores)
print(precision_scores)
print(recall_scores)

# Formatage et sauvegarde des résultats
results_df = pd.DataFrame()
file_path = '/Users/nathanjacas/Documents/Cours/AM/Avaliaçao Comparativa/Comparaison_utility/model_performance.csv'  # Modifiez le chemin selon votre environnement

# Récupérer les scores
metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
scores_lists = [accuracy_scores, precision_scores, recall_scores, f1_scores, rocauc_scores]


# Créer un DataFrame pour chaque métrique
for metric, scores in zip(metrics, scores_lists):
    for fold_index, score in enumerate(scores):
        new_row = pd.DataFrame({
            'Model': ['LVQ'],
            'Metric': [metric],
            'Fold': [fold_index + 1],
            'Score': [score]
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Vérifier si le fichier existe et s'il contient du contenu
header = not os.path.exists(file_path) or os.stat(file_path).st_size == 0

# Écriture dans un fichier CSV
results_df.to_csv(file_path, mode='a', header=header, index=False)





Tamanho do vetor: 113999
Tamanho do vetor 100% 113999
[0.008552631578947369, 0.008771929824561403, 0.008771929824561403, 0.008698830409356725, 0.008845675853498063]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]


In [ ]:
import pandas as pd
from scipy.stats import wilcoxon, friedmanchisquare

# Chargement des données
data = pd.read_csv('/Users/nathanjacas/Documents/Cours/AM/Avaliaçao Comparativa/Comparaison_utility/model_performance.csv')

# Initialiser un DataFrame pour stocker les résultats des tests
test_results = pd.DataFrame(columns=['Metric', 'Comparison', 'Test', 'Statistic', 'P-Value'])

# Boucle sur chaque métrique
for metric in data['Metric'].unique():
    metric_data = data[data['Metric'] == metric]
    
    # Collecter les scores pour chaque modèle et métrique
    models_scores = {model: list(metric_data[metric_data['Model'] == model]['Score']) for model in metric_data['Model'].unique()}
    
    # Comparaisons deux à deux avec le test de Wilcoxon
    models = list(models_scores.keys())
    for i in range(len(models)):
        for j in range(i + 1, len(models)):
            scores1 = models_scores[models[i]]
            scores2 = models_scores[models[j]]
            stat, p = wilcoxon(scores1, scores2)
            new_row = pd.DataFrame({
                'Metric': [metric],
                'Comparison': [f"{models[i]} vs {models[j]}"],
                'Test': ['Wilcoxon'],
                'Statistic': [stat],
                'P-Value': [p]
            })
            test_results = pd.concat([test_results, new_row], ignore_index=True)
    
    # Test de Friedman pour toutes les métriques si plus de 2 modèles
    if len(models) > 2:
        friedman_scores = [models_scores[model] for model in models]
        stat_f, p_f = friedmanchisquare(*friedman_scores)
        new_row = pd.DataFrame({
            'Metric': [metric],
            'Comparison': ['All models'],
            'Test': ['Friedman'],
            'Statistic': [stat_f],
            'P-Value': [p_f]
        })
        test_results = pd.concat([test_results, new_row], ignore_index=True)

# Affichage des résultats
print(test_results)

# Optionnel : Sauvegarder les résultats dans un fichier CSV
test_results.to_csv('test_results.csv', index=False)


       Metric            Comparison      Test  Statistic   P-Value
0    accuracy  SVM vs Decision Tree  Wilcoxon        0.0  0.001953
1    accuracy            SVM vs KNN  Wilcoxon        0.0  0.001953
2    accuracy  Decision Tree vs KNN  Wilcoxon        0.0  0.001953
3    accuracy            All models  Friedman       20.0  0.000045
4   precision  SVM vs Decision Tree  Wilcoxon        0.0  0.001953
5   precision            SVM vs KNN  Wilcoxon        0.0  0.001953
6   precision  Decision Tree vs KNN  Wilcoxon        0.0  0.001953
7   precision            All models  Friedman       20.0  0.000045
8      recall  SVM vs Decision Tree  Wilcoxon        0.0  0.001953
9      recall            SVM vs KNN  Wilcoxon        0.0  0.001953
10     recall  Decision Tree vs KNN  Wilcoxon        0.0  0.001953
11     recall            All models  Friedman       20.0  0.000045
12         f1  SVM vs Decision Tree  Wilcoxon        0.0  0.001953
13         f1            SVM vs KNN  Wilcoxon        0.0  0.00

/var/folders/qg/_cqjp2wj1_j60qh8lvljg5140000gn/T/ipykernel_44879/1226855431.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test_results = pd.concat([test_results, new_row], ignore_index=True)
